<h1>Computer Vision</h1>
<h2>Setup</h2>
<h3>Libraries</h3>

In [24]:
#Basic
import numpy as np
import pandas as pd
import math
#ML/AI
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
#Graphics
import matplotlib.pyplot as plt
from matplotlib import colors

<h3>Variables</h3>

In [25]:
path = ""
AUTOTUNE = tf.data.experimental.AUTOTUNE

<h2>Classification</h2>
<h3>Functions</h3>

In [17]:
def modelCreation(inputShape):
    model = keras.Sequential([
        #Input
        layers.InputLayer(input_shape = inputShape),
        #Augment
        preprocessing.RandomRotation(factor = 0.1),
        #Layer 1
        layers.BatchNormalization(renorm = True),
        layers.Conv2D(filters=10, kernel_size=3, activation='relu', padding='same'),
        #Layer 2
        layers.BatchNormalization(renorm = True),
        layers.Conv2D(filters=20, kernel_size=3, activation='relu', padding='same'),
        #Head
        layers.BatchNormalization(renorm = True),
        layers.Flatten(),
        layers.Dense(8, activation='relu'),
        layers.Dense(1, activation='sigmoid'),
    ])
    optimiser = keras.optimizers.SGD(lr=0.01, nesterov=True)
    model.compile(
        optimizer=optimiser,
        loss='binary_crossentropy',
        metrics=['binary_accuracy'],
    )
    return model
    
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

<h3>Model</h3>

In [15]:
model = modelCreation([128, 128, 3])
early_stopping = keras.callbacks.EarlyStopping(
                    patience=10,
                    min_delta=0.001,
                    restore_best_weights=True,
                 )

<h3>Data Processing</h3>

In [20]:
ds_train_ = image_dataset_from_directory(
            path +'/train',
            labels='inferred',
            label_mode='binary',
            image_size=[128, 128],
            interpolation='nearest',
            batch_size=64,
            shuffle=True,
        )
ds_valid_ = image_dataset_from_directory(
            path +'/valid',
            labels='inferred',
            label_mode='binary',
            image_size=[128, 128],
            interpolation='nearest',
            batch_size=64,
            shuffle=False,
        )
        
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/train'

<h3>Machine Learning</h3>

In [ ]:
his = model.fit(
        ds_train,
        validation_data=ds_valid,
        epochs = 50,
        callbacks=[self.early_stopping],
      )

<h2>Graphics</h2>

In [26]:
def plotImage(image):
    if image.ndim == 2:
        plt.imshow(image, interpolation='nearest')
    else:
        phi = (1 + 5 ** 0.5) / 2
        n = image.shape[0]
        x = (n / phi) ** 0.5
        pdim = [math.ceil(x), math.floor(x * phi)]
        if pdim[0] * pdim[1] < n:
            pdim[1] += 1
        for i in range(n):
            plt.subplot(pdim[0], pdim[1], i + 1)
            plt.imshow(image[i], interpolation='nearest')
    plt.show()